# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [46]:
# Dependencies and Setup
import hvplot.pandas
import holoviews as hv
from holoviews import opts
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [47]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kavieng,-2.5744,150.7967,27.73,77,100,5.38,PG,1731257035
1,1,shimo-furano,43.3500,142.3833,8.96,96,99,3.43,JP,1731257463
2,2,saint-pierre,-21.3393,55.4781,25.82,69,0,1.54,RE,1731257464
3,3,port-aux-francais,-49.3500,70.2167,4.39,81,39,8.85,TF,1731257221
4,4,bredasdorp,-34.5322,20.0403,16.97,67,99,4.08,ZA,1731257466
...,...,...,...,...,...,...,...,...,...,...
91,91,greenville,35.6127,-77.3663,21.19,65,0,3.09,US,1731257309
92,92,luderitz,-26.6481,15.1594,17.45,70,0,10.25,NaN,1731257567
93,93,hermanus,-34.4187,19.2345,18.22,70,100,5.22,ZA,1731257568
94,94,pangani,-5.4333,38.9667,25.74,83,27,4.24,TZ,1731257570


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [49]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles='OSM', 
    size='Humidity', 
    color='City', 
    alpha=0.5, 
    hover_cols=['City', 'Humidity']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [51]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 11) & (city_data_df["Humidity"] < 50)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,bardai,11.6500,7.2333,24.30,29,67,3.00,NG,1731257474
12,12,palmer,42.1584,-72.3287,12.73,44,83,1.34,US,1731257071
36,36,crane,31.3974,-102.3501,19.28,16,0,3.52,US,1731257505
45,45,as sulayyil,20.4607,45.5779,24.47,29,0,4.19,SA,1731257172
61,61,siwa oasis,29.2041,25.5195,21.59,38,0,2.70,EG,1731257533
65,65,punta arenas,-53.1500,-70.9167,11.06,43,40,10.29,CL,1731257511
72,72,ar rutbah,33.0381,40.2848,17.52,42,1,4.84,IQ,1731257545


### Step 3: Create a new DataFrame called `hotel_df`.

In [53]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,bardai,NG,11.6500,7.2333,29,
12,palmer,US,42.1584,-72.3287,44,
36,crane,US,31.3974,-102.3501,16,
45,as sulayyil,SA,20.4607,45.5779,29,
61,siwa oasis,EG,29.2041,25.5195,38,
65,punta arenas,CL,-53.1500,-70.9167,43,
72,ar rutbah,IQ,33.0381,40.2848,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [55]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
filter = f"circle:{longitude},{latitude},{radius}" 
bias = f"proximity:{longitude},{latitude}"  
limit = 1
params = {
    "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key,
    "filter": filter,
    "bias":bias
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bardai - nearest hotel: No hotel found
palmer - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
siwa oasis - nearest hotel: Flower of the city hotel
punta arenas - nearest hotel: Hotel Lacolet
ar rutbah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
10,bardai,NG,11.6500,7.2333,29,No hotel found
12,palmer,US,42.1584,-72.3287,44,No hotel found
36,crane,US,31.3974,-102.3501,16,No hotel found
45,as sulayyil,SA,20.4607,45.5779,29,No hotel found
61,siwa oasis,EG,29.2041,25.5195,38,Flower of the city hotel
65,punta arenas,CL,-53.1500,-70.9167,43,Hotel Lacolet
72,ar rutbah,IQ,33.0381,40.2848,42,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [66]:
%%capture --no-display

# Configure the map plot
hotel_map = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles='OSM', 
    size='Humidity', 
    color='City', 
    alpha=0.5, 
    hover_cols=["Humidity", "Country"]
)

# Display the map
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)